In [1]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()

dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 25)
dataset, frequency_list = text_preprocessing.keep_labels(preprocessded_dataset,keep_ratio=0.10)
slice_dataset = text_preprocessing.dataset_slice(dataset,ratio=0.25)

import numpy as np
all_sente = list(slice_dataset['text'])
all_label = np.array(slice_dataset.drop('text', 1))

X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sente, all_label)

[nltk_data] Downloading package reuters to /Users/monk/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
lower_case done
punctuation removed
text cleaning done


In [2]:
from multilab.models import Elmo_Word_Model

config = {
                         'no_of_labels'               :  y_train.shape[1],
                         'learning_rate'              : 0.001,
                         'rnn_units'                  : 100,
                         'epoch'                      : 1,
                         'batch_size'                 : 128,
                         'dropout'                    : 0.2,
                         'output_type'                : 'state_output',
                         'train_elmo'                 : True,
                         'result_path'                : '/Users/monk/Desktop',
                        }


el_m = Elmo_Word_Model(X_train, y_train, X_test,  y_test, config)
el_m.train()

TypeError: Can't convert 'tokens': Shape TensorShape([Dimension(None)]) is incompatible with TensorShape([Dimension(None), Dimension(None)])

In [ ]:
print(el_m.train())

In [ ]:
# testing

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub

ins = tf.placeholder(tf.string, (None), name='pl_phrase_text')


module = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)
embeddings = module(dict(text=ins))
embeddings = tf.expand_dims(embeddings, axis=1)

In [ ]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    print(sess.run(embeddings,feed_dict={ins:['hello']}).shape)

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub

tokens = tf.placeholder(tf.string, (None), name='pl_phrase_text')
pl_phrase_len = tf.placeholder(tf.int32, (None), name='pl_phrase_len')

tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]


module = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)
module_features = module(dict(tokens=tokens, sequence_len = pl_phrase_len),
                                 signature='tokens', as_dict=True)
embeddingse = module_features["elmo"]

with tf.variable_scope('Layer1'):
    cell_fw1 = tf.nn.rnn_cell.LSTMCell(num_units=128, state_is_tuple=True)
    cell_bw1 = tf.nn.rnn_cell.LSTMCell(num_units=128, state_is_tuple=True)

    outputs1, states1 = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=cell_fw1,
        cell_bw=cell_bw1,
        inputs=embeddingse,
        dtype=tf.float32)

In [ ]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    out, data,t = sess.run([outputs1,embeddingse,module_features],feed_dict={tokens: tokens_input, pl_phrase_len: tokens_length})
    
    
    
    

In [ ]:
rnn_output = tf.reshape(outputs1[0], (-1, 128))
rnn_output.shape

In [ ]:
rnn_outputs = tf.reshape(outputs1[0], (-1, 128 * 6))

In [ ]:
rnn_outputs.shape

In [ ]:
from tqdm import tqdm
import nltk

# padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]


sentences = ['check this out','i am very happy', 'what is', ['check','this'],'he a a a abs']


def pad_sentences(self, sentences, sequence_len):
    padded_sentences = []
    actual_length    = []


    for sentence in tqdm(sentences):
        if not isinstance(sentence, list):
            token = nltk.word_tokenize(sentence)
        else:
            token = sentence
        
        if len(token) < sequene_len:
            actual_length.append(len(token))
            token = token + [''] * (sequene_len-len(token))
        else:
            actual_length.append(len(token))
        
        padded_sentences.append(token)
    return padded_sentences, actual_length

In [ ]:
chy = pad_sentences(sentences,5)

In [ ]:
chy

In [ ]:
actual_len = []
for m in chy[0]:
    actu = [k for k in m if k!='']
    actual_len.append(len(actu))
    

In [ ]:
import numpy as np

def actual_len(list_data):
    
    actual_ = []
    for sequence in list_data:
        actual = [sub_ for sub_ in sequence if sub_!='']
        actual_.append(len(actual))
    return actual_
        
    
    

In [ ]:
data        = actual_len(chy[0])
second_list = actual_len(chy[0])
data.extend(second_list)